# ML pipeline: Binary Classification 

##  <font color='orange'>Using PyCaret</font>

C Fanara, October 2021

BiC_PyCaretv1.1.ipynb - VERBOSE VERSION

***
### 1. Introduction
We use several binary classifiers for benchmark purposes.
We commpare this flow with the protocol within binary_classif-V1.4b.ipynb hence we use the dsame dataset, heart.csv rather than tghe example shown at: 
https://www.datasource.ai/uploads/624e8836466a40923b64b901b5050c0f.html?utm_medium=social&utm_source=linkedin&utm_campaign=postfity&utm_content=postfity447df

For data exploration we use profilers, but dela with those in separate notebook - or the evolution of this one.
For shap, most likely - we hang on until tuning - we'll see

Also for now, we focus on an individual dataset - the dev of a use case - but for extensive bench tasks (LAB) it will be put into an autoML solution to test the several hundreds of datasets (like Penn and UCI collections) - ongoing.
Note - here we learn also the lab management as per instruction at: https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html
***
#### COLOR in markdown
/home/carlo/anaconda3/share/jupyter/lab/themes/@jupyterlab/theme-dark-extension/index.css" [file name index.css] to change the color of text in  section "#content font", 
specifically in "#Defaults use Material Design specification" changing the color code of "jp-content-font-color1. 

May also use a self-made css - see cell at the end - Appendix

***

#### This problem
75 predictor variables, 'age', 'sex','smoker or not' etc, and target variable: presence (1) / absence  (0) of heart disease
Note, the UCI dataset link: https://archive.ics.uci.edu/ml/datasets/Heart+Disease
The main paper: _Detrano, R., Janosi, A., Steinbrunn, W., Pfisterer, M., Schmid, J., Sandhu, S., Guppy, K., Lee, S., & Froelicher, V. (1989). International application of a new probability algorithm for the diagnosis of coronary artery disease. American Journal of Cardiology, 64,304--310
Some meaning - see appendix for the data dictionary_ (web link at UCI not working
Also note the study at: https://towardsdatascience.com/heart-disease-uci-diagnosis-prediction-b1943ee835a7
***

October 2021 


***
v 1.0 <=12 October 
v1.1, rev. 13 October 2021: with optimized plots + interpretation.



<font color='green'>See Todo's list at end</font>
___________________________________________________________

<div class="alert alert-block alert-info">
<b>Note:</b> This is an extended version with tips and discussion, to be continuously enriched as long as we go - and useful for blogging. The production one will be streamlined and less verbose!.
</div>

### 2. Data - import and look
Import DataSet to Panda's DataFrame and perform quick EDA 'by hand'.
Next, we may use profilers, e.g. one or more of (see below):
 - pandas prodiling
 - EDA 
 - Sweetvz
 
Although, in the true pipeline we split the latter pahse form the modeling - as done with the kaggle airbnb, with EDA and modeling (regression) kept into two separated notebooks

In [1]:
import pandas as pd

# might ask filename with interactive  UI
namef='heart.csv' # the bare name

# read csv using pandas
df = pd.read_csv('~/Documents/Infra_py-ML/classif_data/heart.csv')

# display basic information about the dataframe
print('Dataframe\'s shape: {}\n'.format(df.shape))

# for later (write at the end)
tsk='BiCl'  # others might be: "MuCl"=Multi-class; "Regr"=Regression

df.info()
# describe numeric columns
# df.describe()

df.head(10)

Dataframe's shape: (303, 14)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


Here we sample 5% of the data which will be labelled as 'unseen' in order to spare for the prediction.

In [2]:
## sample returns a random sample from an axis of the object. That would be 22,800 samples, not 24,000
data = df.sample(frac=0.95, random_state=786)
data.shape


(288, 14)

In [3]:
# we remove from the original dataset this random data: the unseen is the validation set
data_unseen = df.drop(data.index)
data_unseen.shape

(15, 14)

In [4]:
## we reset the index of both datasets
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (288, 14)
Unseen Data For Predictions: (15, 14)


### 3. Data splitting

Put simple figure for the representation of the data, including the split
(demo/edu purposes)

![image.png](attachment:xxxxxx.png)

 <font color='red'>CAUTION: Actually we started with 288 records and we get only 14 left for the predictions...</font>

Summarizing from original link (see above):

- _**unseen**_: is the the validation data set used to evaluate the model, but only once the latter is fully trained (using the training and test sets).

- _**train**_: used to train the model which sees and learns from this data

- _**test**_: provides unbiased assessment of a model, matched to the training set while adjusting the hyperparameters. This is for frequent evaluation 

So, it will be interesting to see what the models (18 in total) will tell us given the limited number of rows.


### 4. Setup and Train
pyCaret uses the setup() function with the purpose to:
- initialize the einvironment
- create the transofrmation pipeline for training and deployment

It must be the first function called within pycaret and takes at miimum 2 params: pandas df and target column 
(our X and y in our protocol)

In [5]:
## setting up the environment
from pycaret.classification import *

Most of the parameters can be customized. For us it might be interesting to see if standardization is avoidable - the tutorial does it automatically
(in their example, the categories are hot coded, hence the number of column raises from 24 to 91).
Check parameters from: 

In [6]:
# press enter to finish!
# model_setup = setup(data=data, target='target', session_id=100)
# model_setup = setup(data=data, target='target', session_id=None) # session id to None by default. Arguably reproducible?
model_setup = setup(data=data, target='target') # session id to None by default. Arguably reproducible?

# note, both in a python 3.6 and python 3.9.6 based kernels we get a recurrent error - apparently unrelated  AttributeError - always after step 59 SMOTE - It is NOT 
# blocking (can get on until end of the pipeline...) 
# same error repeats upon write. Issue was submitted to Pycaret folks on 12 oct 21

,Description,Value
0,session_id,6478
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(288, 14)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


<font color='red'>CAUTION: </font>
Using two different kernels corresponding to python 3.6.2 and 3.9.6 python versions in their respective conda environmentst, I get the error:

    Apparently, the only way to solve this would be to install and use a purely 'oycaret env' as per:
    https://stackoverflow.com/questions/69106929/pycaret-anomaly-detection-setup-valueerror-setting-a-random-state-has-no-effec

**Note** pycaret use N-fold CV. This may be particularly useful when data are limited - like here a 70:30 - may lead to large variance especially as in the test, we only have 15 records!

**Note** the sequence of actions here - different from other protocol (cf notebook) in the use of the validation set. 
This is the unseen data extracted and held out before. 

The procedure: randomly select observations into k 'folds' (groups) of ~same size. 
The first fold is the instantaneous validation set, and the method is trained on the remaining k − 1 folds. Then the procedure is repeated by taking the second fold as the validation, and training on the remaining k-1 folds. The process is repeated until all folds have been processed and the metric evaluated by averaging over the iterations.

Hence, this is a **stratified** cross-validation (see appendix for discussion of k) where the metrics are obtained by calling to the function _best_model()_ , which trains and compares all models in the pyCaret model library but also models available in the environment used (conda here).

Note. Worth discussing in depth the effect of k in k-fold (the Variance - due to the limited sample size should dominate over model uncertainty). 

Note on setup initialization: data types can be defined programmatically rather than through the user input box (see regression example).
The 'silent' parameter is set to True to avoid input; the _categorical_features_ takes the name of categorical columns as string, and _numeric_features_ takes the name of numeric columns as a string:

In [7]:
#reg1 = setup(data, target = 'charges', silent=True, 
#            categorical_features=['sex', 'smoker', 'region', 'children'], 
#            numeric_features=['age', 'bmi'])

Also. to drop some variables from the modeling (that is without deleting them from pandas df), in setup just add:
ignore_features = ['Country Name', 'Indicator Name'] (as example)

In Binary Classification, when optimizing a solution for a business problem where Type 1 and Type 2 errors have a different impact (consider health where the importance of FP and TP may have dire consequences...), the classifier may beoptimized for a probability threshold value - to optimize the custom loss function, by, for example:
1) defining the cost of true positives, true negatives, false positives and false negatives separately. By default, all classifiers have a threshold of 0.5.
2) use the optimize threshold function for trained model
optimize_threshold(xgboost, true_negative = 1500, false_negative = -5000).

This might be the case in a threshold based logistic regression problem (scores, risks).

Below we train /compare the available models

In [8]:
# best_model = compare_models() # basic
# best_model = compare_models(sort = 'AUC', fold = 5)
# best_model=compare_models(include=['lr', 'rf', 'dt'...] # can select some and including non sk-learn based
best_model = compare_models(sort = 'AUC', fold = 5, n_select=5) # n_select works by providing the ;list of the top k models - no need to retrain
# use n_select to limit to the first 'k' models - not all 13...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8409,0.9183,0.8714,0.8459,0.8568,0.6775,0.6813,0.6440
et,Extra Trees Classifier,0.8655,0.9172,0.8801,0.8771,0.8764,0.7282,0.7324,0.2280
rf,Random Forest Classifier,0.8357,0.9165,0.8805,0.8303,0.8526,0.6668,0.6733,0.2500
lda,Linear Discriminant Analysis,0.8606,0.9150,0.9078,0.8497,0.8766,0.7164,0.7213,0.0120
gbc,Gradient Boosting Classifier,0.8109,0.8856,0.8528,0.8115,0.8294,0.6164,0.6232,0.0420
lightgbm,Light Gradient Boosting Machine,0.8359,0.8806,0.8433,0.8536,0.8471,0.6693,0.6714,0.0280
nb,Naive Bayes,0.8111,0.8623,0.8892,0.7922,0.8364,0.6138,0.6233,0.0120
ada,Ada Boost Classifier,0.7263,0.8096,0.7606,0.7398,0.7494,0.4472,0.4487,0.0480
dt,Decision Tree Classifier,0.7313,0.7272,0.7602,0.7558,0.7502,0.4565,0.4688,0.0120
knn,K Neighbors Classifier,0.6172,0.6541,0.6792,0.6379,0.6565,0.2240,0.2260,0.4280


Note that we added extreme boosting - xgb - and catboost, not originally in the python conda environment using a conda install. So now we have 15 algorithms.
Note: could improve the table, by adding a calculated column like overfitting (difference bwetween test and train "error") but plotting functions below allow for this

In [9]:
top3 = compare_models(sort = 'AUC', fold = 5, n_select=3) # n_select works by providing the ;list of the top k models - no need to retrain
print(top3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8409,0.9183,0.8714,0.8459,0.8568,0.6775,0.6813,0.0980
et,Extra Trees Classifier,0.8655,0.9172,0.8801,0.8771,0.8764,0.7282,0.7324,0.2300
rf,Random Forest Classifier,0.8357,0.9165,0.8805,0.8303,0.8526,0.6668,0.6733,0.2480
lda,Linear Discriminant Analysis,0.8606,0.9150,0.9078,0.8497,0.8766,0.7164,0.7213,0.0120
gbc,Gradient Boosting Classifier,0.8109,0.8856,0.8528,0.8115,0.8294,0.6164,0.6232,0.0400
lightgbm,Light Gradient Boosting Machine,0.8359,0.8806,0.8433,0.8536,0.8471,0.6693,0.6714,0.0240
nb,Naive Bayes,0.8111,0.8623,0.8892,0.7922,0.8364,0.6138,0.6233,0.0120
ada,Ada Boost Classifier,0.7263,0.8096,0.7606,0.7398,0.7494,0.4472,0.4487,0.0500
dt,Decision Tree Classifier,0.7313,0.7272,0.7602,0.7558,0.7502,0.4565,0.4688,0.0120
knn,K Neighbors Classifier,0.6172,0.6541,0.6792,0.6379,0.6565,0.2240,0.2260,0.0580


[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6478, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=6478, verbose=0,
                     warm_start=False), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_featur

You could use these models to generate diagnostic plots or to even use them for predictions, if you would like"
predict_model(top3[0], data=data) 

In [10]:
print(best_model)

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6478, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=6478, verbose=0,
                     warm_start=False), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_featur

In [11]:
# models() # just to look among the available - could put earlier above

### 5. Tuning

**Notes**
- Tune_model() automatically tunes the hyperparameters of a model using the Random Grid Search in a predefined search space. Can we use a different method?
e.g. set up a Simulated annealing and a Bayesian Optimization
According to handbook: To use a custom search grid, pass the custom_grid parameter in the tune_model function. 
We'll try several iterations

- Also, can increase the n_iter parameter to improve the performance, see commented instructions below - but training time may be high.

- Couldn't we pass the best model directly to tune_model() function?

- why do I need to regenerate it? (e.g. call create_model: can't i just select the best ranked and pass it to tune_model()?

#### <font color='red'>CAUTION</font>
I keep here the attribution of the explicit name of the winner model: its name is assigned to the generic 'model', e.g. the name referred to below (otherwise for the following we would need specifying explicitly which one (Until we get a way to intercept and pass directly to _tune_model_)

In [12]:
## our best model was lr but we can't just select "best_model", we have to generate it and thereafter tune it. Could be more streamlined here?
#model=create_model(best_model) # error: ...does not have the fit method
model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8095,0.8545,0.9091,0.7692,0.8333,0.6147,0.6264
1,0.9500,0.9596,1.0000,0.9167,0.9565,0.8980,0.9027
2,0.8000,0.9394,1.0000,0.7333,0.8462,0.5789,0.6383
3,0.8000,0.8788,0.8182,0.8182,0.8182,0.5960,0.5960
4,0.7500,0.9091,0.7273,0.8000,0.7619,0.5000,0.5025
5,0.9000,0.9495,0.9091,0.9091,0.9091,0.7980,0.7980
6,0.9000,0.9798,1.0000,0.8462,0.9167,0.7938,0.8112
7,0.8500,0.8485,0.8182,0.9000,0.8571,0.7000,0.7035
8,0.8500,0.9293,0.8182,0.9000,0.8571,0.7000,0.7035
9,0.9500,0.9500,0.9000,1.0000,0.9474,0.9000,0.9045


In [13]:
# tuned_mod = tune_model(model)
# custom_grid =?
# optimize = 'AUC'
# tune with n_iter = 50
tuned_mod = tune_model(model, n_iter = 10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8571,0.8636,0.9091,0.8333,0.8696,0.7123,0.7156
1,0.9500,0.9798,1.0000,0.9167,0.9565,0.8980,0.9027
2,0.8500,0.9394,1.0000,0.7857,0.8800,0.6875,0.7237
3,0.7500,0.8283,0.7273,0.8000,0.7619,0.5000,0.5025
4,0.7500,0.8788,0.7273,0.8000,0.7619,0.5000,0.5025
5,0.9000,0.9495,0.9091,0.9091,0.9091,0.7980,0.7980
6,0.9000,0.9798,1.0000,0.8462,0.9167,0.7938,0.8112
7,0.8500,0.8384,0.8182,0.9000,0.8571,0.7000,0.7035
8,0.8500,0.9293,0.8182,0.9000,0.8571,0.7000,0.7035
9,0.9500,0.9500,0.9000,1.0000,0.9474,0.9000,0.9045


In [14]:
print(tuned_mod)

LogisticRegression(C=3.315, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6478, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [15]:
print(model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6478, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


Notes
- that by default, tune_model optimizes Accuracy but we may change (see above, ex, optimize = 'AUC').
- that when the data set is unbalanced Accuracy might be not the best metric to consider. Discuss
- the grid containing score can be puled into a dataframe so we can save a model and its metric into a file fro checks and docs(this can be done for all the training functions (create, tune, ensemble...) 

NOTEWORTHY: you can create a loop to train a model with different parameters and create a comparison table - for example looping along a learning rate or a number of tress etc, and produce a results table, see https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html


### 6. Model plots and Evaluation

plot_model() function is used to analyze the performance using AUC, confusion_matrix, decision boundary ... by taking a trained model object and returning a graph based on the training/test set.
In previous version 1.0 - we chose among the 15 different plots available, cf plot_model() docs). Here we limit to the global and interactive evaluate_model, see below
Note, we may use a double call using the original, 'model' and the tuned, 'tuned_mod' - but don't know how to do this side by side

In [17]:
evaluate_model(tuned_mod)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

(worth explaining gain and lift charts for the given problem)

### 7. Interpretation
SHAP is used in summary, correlation and reason plots. Other plots use different methods.
Using the defaults in pyCaret (see also notebook Binary protocol for other plots applied to specific models)

By default the function uses the named model on the test data

#### NB
The function only supports tree based models
(May be worth selecting just tree models when testing)

In [18]:
#interpret_model(tuned_mod)  # tuned mod has to be one of the trre models

### 8. Finalize
Purpose: get final version before production. The finalize_model() function fits the model to the **entire** data set, e.g. train and test (adding the 30% in this case). One may execute this _after_ or _before_ the prediction (via the predict_model() function). We're going to execute it _before_ prediction

In [19]:
final_mod = finalize_model(tuned_mod)

**NB** Once the model is finalized using finalize_model(), the _entire data set_ was used for training. So it makes no sense to make predictions about the test set after finalize_model() - it is like making predictions on the same data that was used for the modeling! 

### 9. Predict
A final check is done by predicting the test/hold-out set (_data_unseen_ in our case, recall, 30% of the data was separated as training/set samples).

All the metrics seen above are cross-validated results from the training set (70%), using our final training model stored in _tuned_mod_ (this example) we predict against the test sample and evaluate the metrics to see if they are substantially different from the CV results.

In [20]:
predict_model(final_mod)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8736,0.9486,0.9000,0.8824,0.8911,0.7404,0.7406


,age,trestbps,chol,thalach,oldpeak,sex_1,cp_0,cp_1,cp_2,cp_3,...,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3,target,Label,Score
0,39.0,118.0,219.0,140.0,1.2,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.8823
1,52.0,134.0,201.0,158.0,0.8,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0.6270
2,60.0,130.0,206.0,132.0,2.4,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.9976
3,48.0,122.0,222.0,186.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0.8822
4,61.0,134.0,234.0,145.0,2.6,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0.7891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,41.0,120.0,157.0,182.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0.9738
83,56.0,120.0,236.0,178.0,0.8,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0.9582
84,45.0,115.0,260.0,185.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0.8742
85,76.0,140.0,197.0,116.0,1.1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0.9235


The increase in AUC: 93.38 (mod train); 93.68 (tuned); 96.67 (test/holdout)
The increase in accuracy:: .8802 (mod train); .8752 (tuned); .09080 (test/holdout)

So we are overfitting - quite expected, especially because of the data paucity.

Proceed with the completion of the model and the prediction on the unseen data (the 5% that we had left out at the start and that was never seen by PyCaret)

(TIP: look at the standard deviation of the results of the training set when using create_model() - todo)

In [21]:
unseen_predictions = predict_model(final_mod, data=data_unseen)
# unseen_predictions.head()
unseen_predictions # look at all, they are few!

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,Label,Score
0,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1,0,0.5663
1,59,1,0,135,234,0,1,161,0,0.5,1,0,3,1,0,0.8168
2,62,1,2,130,231,0,1,146,0,1.8,1,3,3,1,0,0.9375
3,46,1,1,101,197,1,1,156,0,0.0,2,0,3,1,1,0.9174
4,42,1,2,120,240,1,1,194,0,0.8,0,0,3,1,1,0.8896
5,42,1,1,120,295,0,1,162,0,0.0,2,0,2,1,1,0.9310
6,60,1,0,117,230,1,1,160,1,1.4,2,2,3,0,0,0.9413
7,58,1,0,150,270,0,0,111,1,0.8,2,0,3,0,0,0.9273
8,43,0,0,132,341,1,0,136,1,3.0,1,0,3,0,0,0.8968
9,55,1,0,140,217,0,1,111,1,5.6,0,0,3,0,0,0.9871


### 10. Score
Note the last column generated above: 'Label' is the prediction, 'Score' is the corresponding probability. 
Only one data point out of the 15 (0-14) has a probability near 50% (the first).

Note that the predicted results are concatenated with the **original data set** while all the transformations are automatically performed in the background

LogisticRegression(C=3.315, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6478, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

When new data come i: PyCaret's save_model() allows saving the model along with all the transformation pipe for later use and it is stored in a Pickle in the local environment. 

### 11. Save for production

In [31]:
# save_model(lr, 'datasets/Final Model 13 Oct 2021')
save_model(final_mod, 'Final Model 13 Oct 2021')
# worth looking at the flow management and the experiment organization, check MLflow?


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='target',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegression(C=3.315, class_weight='balanced',
                     

### 12. Load model for use

To load a model saved at a future date in the same or an alternative environment, we would use PyCaret's load_model() function and then easily apply the saved model to new unseen data for the prediction

In [33]:
saved_final_mod = load_model('Final Model 13 Oct 2021')

Transformation Pipeline and Model Successfully Loaded


Once the model is loaded into the environment, it can simply be used to predict any new data using the same predict_model() function. Next we have applied the loaded model to predict the same data_unseen we used before.

In [34]:
new_prediction = predict_model(saved_final_mod, data=data_unseen)
new_prediction.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,Label,Score
0,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1,0,0.5663
1,59,1,0,135,234,0,1,161,0,0.5,1,0,3,1,0,0.8168
2,62,1,2,130,231,0,1,146,0,1.8,1,3,3,1,0,0.9375
3,46,1,1,101,197,1,1,156,0,0.0,2,0,3,1,1,0.9174
4,42,1,2,120,240,1,1,194,0,0.8,0,0,3,1,1,0.8896


## Appendices

#### Appendix 1 - todo's list
- implement a modified pipeline to test a only a few selected models but accross a set of (possibly numerous) datasets:
    - with idea of using autoML like approach
    - use method like in other protocol (list of estimators, click/select/able via widget UI 
    - (or use external text parameter file)
    
- test the ensemble, blend and stack functions (these exist for classification and regression alike)

- transform in app: streamlit possible?

- for some models (only some) cuda is possible: https://stackoverflow.com/questions/67579616/pycaret-methods-on-gpu-tpu
https://openbase.com/python/pycaret/documentation#pycaret-on-gpu

#### Appendix 2 - about k in CV
There appear to be a numberof choices for k:

- Representative: k is chosen such that each train/test group of data samples is large enough to be statistically representative of the broader dataset (still a bit qualitative, but one can invent a value which is the threshold defining a sample ratio to the total nr of observations.

- k=10: a value found through experimentation to often result in a model estimate with low bias an limited variance: couldd we define a metric bias v variancce - like a ratio or so?

- k=n: k is fixed at n, the size of the dataset: we give each sample the chance of being used the hold-out dataset, the LOO-CV (leave-one-out cross-validation).


#### Appendix 3 Config of background

Two functions in PyCaret _get_config_ and _set_config_ allow to access and change everything in the background, from the training set to the random state etc, cf documentation of get_config function by simply calling help(get_config) as below:

In [50]:
help(get_config)


Help on function get_config in module pycaret.classification:

get_config(variable:str)
    This function retrieves the global variables created when initializing the 
    ``setup`` function. Following variables are accessible:
    
    - X: Transformed dataset (X)
    - y: Transformed dataset (y)  
    - X_train: Transformed train dataset (X)
    - X_test: Transformed test/holdout dataset (X)
    - y_train: Transformed train dataset (y)
    - y_test: Transformed test/holdout dataset (y)
    - seed: random state set through session_id
    - prep_pipe: Transformation pipeline
    - fold_shuffle_param: shuffle parameter used in Kfolds
    - n_jobs_param: n_jobs parameter used in model training
    - html_param: html_param configured through setup
    - create_model_container: results grid storage container
    - master_model_container: model storage container
    - display_container: results display container
    - exp_name_log: Name of experiment
    - logging_param: log_experiment para

Also to inspect the variables in the X set - say upon transformation - one may do:

In [51]:
get_config('X')

,age,trestbps,chol,thalach,oldpeak,sex_1,cp_0,cp_1,cp_2,cp_3,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_1,thal_2,thal_3
0,54.0,110.0,239.0,126.0,2.8,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,64.0,125.0,309.0,131.0,1.8,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,42.0,140.0,226.0,178.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,66.0,146.0,278.0,152.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,42.0,120.0,209.0,173.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,53.0,128.0,216.0,115.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
284,39.0,118.0,219.0,140.0,1.2,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
285,70.0,160.0,269.0,112.0,2.9,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
286,67.0,152.0,212.0,150.0,0.8,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Appendix 4 - Logging experiments

This is the example provided in the turorial - worth noting that MLFlow provides a folder structure and logs/saves all the variables and the models etc. 
Whn fully tested/modifed - use in lab.

Meta data generated by just a few tests is impossible track manually, hence log!
This link https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html provides a tip: 

In [35]:
# loading dataset - uncomment to run example.
from pycaret.classification import *
s = setup(data, target = 'target', silent = True, log_experiment = True, experiment_name = 'heart-1') # compare baseline models
best = compare_models()# generate logs
get_logs()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8462,0.9235,0.8886,0.8595,0.8655,0.6823,0.6982,0.3930
lda,Linear Discriminant Analysis,0.8460,0.9199,0.8970,0.8527,0.8678,0.6805,0.6964,0.0090
ridge,Ridge Classifier,0.8410,0.0000,0.9053,0.8419,0.8660,0.6684,0.6855,0.0090
rf,Random Forest Classifier,0.8410,0.9155,0.8803,0.8578,0.8634,0.6713,0.6819,0.2230
ada,Ada Boost Classifier,0.8260,0.8792,0.8530,0.8621,0.8508,0.6395,0.6529,0.0380
gbc,Gradient Boosting Classifier,0.8257,0.8921,0.8705,0.8434,0.8514,0.6390,0.6511,0.0370
lightgbm,Light Gradient Boosting Machine,0.8257,0.9072,0.8614,0.8496,0.8509,0.6386,0.6489,0.0190
nb,Naive Bayes,0.8210,0.8743,0.8636,0.8430,0.8476,0.6278,0.6386,0.0090
et,Extra Trees Classifier,0.8157,0.8944,0.8788,0.8271,0.8466,0.6159,0.6291,0.2090
dt,Decision Tree Classifier,0.7267,0.7226,0.7508,0.7790,0.7576,0.4425,0.4541,0.0100


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.F1,metrics.Kappa,metrics.Prec.,metrics.AUC,...,tags.Run Time,tags.URI,tags.mlflow.source.name,tags.mlflow.source.type,tags.USI,tags.mlflow.runName,tags.Run ID,tags.mlflow.log-model.history,tags.mlflow.user,tags.Source
0,38bc5efd9b464c668032de161f7122f7,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:58.132000+00:00,2021-10-13 15:40:58.193000+00:00,0.6495,0.2180,0.7542,0.0000,...,0.3,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,SVM - Linear Kernel,38bc5efd9b464c668032de161f7122f7,"[{""run_id"": ""38bc5efd9b464c668032de161f7122f7""...",carlo,compare_models
1,7fd4d7b55fd64c3db9575ea2fc500103,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:57.952000+00:00,2021-10-13 15:40:58.004000+00:00,0.7280,0.3240,0.6915,0.7181,...,0.73,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,K Neighbors Classifier,7fd4d7b55fd64c3db9575ea2fc500103,"[{""run_id"": ""7fd4d7b55fd64c3db9575ea2fc500103""...",carlo,compare_models
2,6706727e838e41c7bd1524fa122bde6d,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:57.767000+00:00,2021-10-13 15:40:57.823000+00:00,0.6967,0.3425,0.7607,0.6701,...,0.31,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Quadratic Discriminant Analysis,6706727e838e41c7bd1524fa122bde6d,"[{""run_id"": ""6706727e838e41c7bd1524fa122bde6d""...",carlo,compare_models
3,4d04a7b2f1bf44c7a03d987f72c72584,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:57.587000+00:00,2021-10-13 15:40:57.642000+00:00,0.7576,0.4425,0.7790,0.7226,...,0.3,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Decision Tree Classifier,4d04a7b2f1bf44c7a03d987f72c72584,"[{""run_id"": ""4d04a7b2f1bf44c7a03d987f72c72584""...",carlo,compare_models
4,adb308118a1a4fe2ab92a3834548f289,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:57.408000+00:00,2021-10-13 15:40:57.464000+00:00,0.8466,0.6159,0.8271,0.8944,...,2.27,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Extra Trees Classifier,adb308118a1a4fe2ab92a3834548f289,"[{""run_id"": ""adb308118a1a4fe2ab92a3834548f289""...",carlo,compare_models
5,40f71955f14845d28fe2e2e35d2d37c9,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:57.231000+00:00,2021-10-13 15:40:57.283000+00:00,0.8476,0.6278,0.8430,0.8743,...,0.3,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Naive Bayes,40f71955f14845d28fe2e2e35d2d37c9,"[{""run_id"": ""40f71955f14845d28fe2e2e35d2d37c9""...",carlo,compare_models
6,2cfc69bc06cd44b88c4b38fc57c5c2bc,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:57.046000+00:00,2021-10-13 15:40:57.104000+00:00,0.8509,0.6386,0.8496,0.9072,...,0.39,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Light Gradient Boosting Machine,2cfc69bc06cd44b88c4b38fc57c5c2bc,"[{""run_id"": ""2cfc69bc06cd44b88c4b38fc57c5c2bc""...",carlo,compare_models
7,216dc738ec2f47dbae84eb7df2803d14,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:56.863000+00:00,2021-10-13 15:40:56.918000+00:00,0.8514,0.6390,0.8434,0.8921,...,0.58,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Gradient Boosting Classifier,216dc738ec2f47dbae84eb7df2803d14,"[{""run_id"": ""216dc738ec2f47dbae84eb7df2803d14""...",carlo,compare_models
8,b779da3340b6400dbd92debd43253945,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/ML_pr...,2021-10-13 15:40:56.680000+00:00,2021-10-13 15:40:56.739000+00:00,0.8508,0.6395,0.8621,0.8792,...,0.59,c9b75d81,/home/carlo/miniconda3/envs/epycaret/lib/pytho...,LOCAL,24b8,Ada Boost Classifier,b779da3340b6400dbd92debd43253945,"[{""run_id"": ""b779da3340b6400dbd92debd43253945""...",carlo,compare_models
9,c7158c395cbb41668fee2ed1b4d45995,1,FINISHED,file:///home/carlo/Documents/Infra_py-ML/

#### Appendix 5 - Markdown color/s in notebook
check the following code

In [36]:
#from IPython.core.display import HTML
#def css_style():
#    styles = open("[path to .css file]","r").read()
#    return HTML(styles)

#css_style()